In [1]:
%pip install -q --disable-pip-version-check semantic-link-labs

StatementMeta(, de1b840a-ebca-4a96-bd96-1d4e75e1e14d, 7, Finished, Available, Finished)

Note: you may need to restart the kernel to use updated packages.



Set Parametners including clone DB name

In [ ]:
import Microsoft.AnalysisServices.Tabular as TOM
import Microsoft.AnalysisServices
import sempy.fabric
import sempy_labs as labs
from sempy import fabric
import sempy
import pandas
import json
import time
import uuid


LakehouseName = "AdventureWorks"
SemanticModelName = f"{LakehouseName}_model"
ClonedModelName = SemanticModelName + "_clone"


workspaceId = "1f8bfacb-b45c-44a7-a487-ac125ab481ac"
lakehouseId = "424a14f9-4a1b-468e-af8d-65add767ad40"


StatementMeta(, de1b840a-ebca-4a96-bd96-1d4e75e1e14d, 92, Finished, Available, Finished)


# Clone AdventureWorks semantic model

In [ ]:
df = sempy.fabric.list_items()
if ClonedModelName in df.values:
    model_id = df.at[df[df['Display Name'] == ClonedModelName].index[0], 'Id']
    fabric.delete_item(model_id)
    print("Cloned model deleted")

with labs.tom.connect_semantic_model(dataset=SemanticModelName, readonly=False) as tom:
    newDB = tom._tom_server.Databases.GetByName(SemanticModelName).Clone()
    newModel = tom._tom_server.Databases.GetByName(SemanticModelName).Model.Clone()
    newDB.Name = ClonedModelName
    newDB.ID = str(uuid.uuid4())
    #newDB.Model = newModel
    newModel.CopyTo(newDB.Model)
    tom._tom_server.Databases.Add(newDB)

    newDB.Update(Microsoft.AnalysisServices.UpdateOptions.ExpandFull)

# Convert cloned model to Direct Lake over SQL Endpoint

In [73]:


with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:


    #Switch to Direct Lake over SQL
    for e in tom.model.Expressions:
        e.Expression = f"""
        let
            Source = Sql.Database("krrvjodmlo3ehogm2woebho6je-zp5iwh24wstujjehvqjfvnebvq.datawarehouse.fabric.microsoft.com", "7c63a301-c62e-408e-9c5e-b7b12b41895a")
        in
            Source"""

        # if e.Name == "DirectLake - AdventureWorks" :
        #     tom.model.Expressions.Remove(e)

print("Converted to Direct Lake over SQL")

StatementMeta(, 3e13bf92-0430-4b5c-ac73-c9d5f7d2363b, 80, Finished, Available, Finished)

Converted to Direct Lake over SQL


# Convert cloned model to Direct Lake over One Lake

In [ ]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:

    for t in tom.model.Tables:
        for p in t.Partitions:
            if(p.Mode==TOM.ModeType.Import):
                t.Partitions.Remove(p)
                tom.add_entity_partition(table_name=t.Name,entity_name=t.Name)
                print(f"Table {t.Name} converted")
            p.Source.SchemaName=None

    for e in tom.model.Expressions:
        e.Expression = f"""
        let
            Source = AzureStorage.DataLake("https://northcentralus-onelake.dfs.fabric.microsoft.com/{workspaceId}/{lakehouseId}", [HierarchicalNavigation=true])
        in
            Source"""
        
print("Converted Import tables back to Direct Lake (on One Lake)")

StatementMeta(, de1b840a-ebca-4a96-bd96-1d4e75e1e14d, 134, Finished, Available, Finished)

Converted Import tables back to Direct Lake (on One Lake)


In [ ]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for e in tom.model.Expressions:
        print(e.Expression)

StatementMeta(, de1b840a-ebca-4a96-bd96-1d4e75e1e14d, 135, Finished, Available, Finished)


        let
            Source = AzureStorage.DataLake("https://northcentralus-onelake.dfs.fabric.microsoft.com/1f8bfacb-b45c-44a7-a487-ac125ab481ac/424a14f9-4a1b-468e-af8d-65add767ad40", [HierarchicalNavigation=true])
        in
            Source


# Show storage mode for each table

In [ ]:
import pandas as pd

objects = {}
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for t in tom.model.Tables:
        #print(t.Name)
        for p in t.Partitions:
            #print(p.Mode)
            objects[t.Name] = str(p.Mode)
 
df=pd.DataFrame([objects])
display(df)

StatementMeta(, de1b840a-ebca-4a96-bd96-1d4e75e1e14d, 132, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3de9d393-c1e7-4614-94a4-71b2ff0dc516)

# Try to convert Direct Lake table to Import
#### Will fail if Direct Lake over SQL

In [ ]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    tom.convert_direct_lake_to_import(
        table_name="DimDate" ,
        entity_name="DimDate" ,
        source="AdventureWorks",
        source_type = "Lakehouse"
    )

StatementMeta(, de1b840a-ebca-4a96-bd96-1d4e75e1e14d, 137, Finished, Available, Finished)

🟢 The 'DimDate' table has been converted to Import mode.


# Refresh composite model so import table gets hydrated

In [ ]:
sempy.fabric.refresh_dataset(ClonedModelName)

StatementMeta(, de1b840a-ebca-4a96-bd96-1d4e75e1e14d, 139, Finished, Available, Finished)

'50deb92e-abb7-422f-b8a3-6cd1a2a25b5e'

In [ ]:
import json
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    x= tom.get_bim()

    formatted_json = json.dumps(x, indent=4)
    print(formatted_json)

StatementMeta(, de1b840a-ebca-4a96-bd96-1d4e75e1e14d, 138, Finished, Available, Finished)

AdventureWorks_model_clone
{
    "name": "AdventureWorks_model_clone",
    "id": "29c8b268-d1f7-4f36-bd16-5d1ea028a051",
    "compatibilityLevel": 1605,
    "model": {
        "culture": "en-US",
        "collation": "Latin1_General_100_BIN2_UTF8",
        "defaultPowerBIDataSourceVersion": "powerBI_V3",
        "tables": [
            {
                "name": "DimCustomer",
                "lineageTag": "5174724d-1f6c-43c5-bd80-87e4fffba36c",
                "sourceLineageTag": "[dbo].[DimCustomer]",
                "columns": [
                    {
                        "name": "CustomerKey",
                        "dataType": "int64",
                        "sourceColumn": "CustomerKey",
                        "lineageTag": "029599fe-ba7f-44a5-a076-018b208a739e"
                    },
                    {
                        "name": "GeographyKey",
                        "dataType": "int64",
                        "sourceColumn": "GeographyKey",
                       